In [1]:
import numpy as np
import pandas as pd
import tarfile

In [2]:
spam_zip = tarfile.open('datasets/spam/spam.bz2')
easy_ham_zip = tarfile.open('datasets/spam/easy_ham.bz2')

In [30]:
mail_list[:9]

[['from', 'Hosting', 'month', 'per'],
 ['month', 'from', 'Hosting', 'per', 'Aug'],
 ['month', 'from', 'Hosting', 'per', 'Aug'],
 ['month', 'from', 'Hosting', 'per', 'Aug'],
 ['month', 'from', 'Hosting', 'Normal', 'per', 'Aug'],
 ['month',
  'Express',
  'from',
  'Hosting',
  'Outlook',
  'Normal',
  'Microsoft',
  'per',
  'Aug'],
 ['month',
  'MimeOLE',
  'Express',
  'from',
  'Hosting',
  'Outlook',
  'Normal',
  'By',
  'Produced',
  'Microsoft',
  'per',
  'Aug'],
 ['month',
  'MimeOLE',
  'Express',
  'from',
  'Hosting',
  'Outlook',
  'Normal',
  'By',
  'Produced',
  'Microsoft',
  'per',
  'Aug'],
 ['month',
  'MimeOLE',
  'Express',
  'from',
  'Hosting',
  'Outlook',
  'Normal',
  'By',
  'Produced',
  'Microsoft',
  'per',
  'Aug']]

In [31]:
def get_mail_list(directories, lab):
    mails = []
    labels = []
    index = 0
    for directory in directories:
            mail_list = tarfile.open(directory)
            for i in mail_list.getmembers()[1:]:
                wordset = set()
                linebyline = str(mail_list.extractfile(i.name).read()).split('\\n')
                for line in linebyline:
                    if 'Subject:' in line:
                        startindex = linebyline.index(line)
                        break
                    else:
                        startindex = 0
                for line in linebyline[startindex:]:
                    for word in line.split(' '):
                        if word.isalpha():
                            wordset.add(word)
                mails.append(list(wordset))
                labels.append(lab[index])
            index += 1
    return(mails, labels)

In [5]:
def turn_to_string(id):
    a = list(open('datasets/spam/spam_ham/mail_'+ str(id) + '.txt', 'r', errors = 'ignore'))
    mail_str = ''
    for line in a:
        if 'Subject:' in line:
            topic_ind = a.index(line)
        else:
            topic_ind = 0
    for line in a[topic_ind:]:
        for char in line.split(' '):
            if char.isalpha() == True:
                mail_str += char + ' '
    return(mail_str)

In [6]:
def generate_word_list(doc):
    word_set = set()
    for mail in doc:
        for word in mail:
            word_set.add(word)
    return(list(word_set))

In [7]:
def tokenize(word_list):
    token_table = {}
    iter0 = 0
    for i in word_list[1:]:
        token_table[i] = iter0
        iter0 += 1
    return(token_table)

In [8]:
def gen_embedded_matr(table, mail_list):
    embedded_list = np.memmap('embed.mmap', dtype = 'float32', shape = (len(mail_list), len(table)), mode = 'write')
    embedded_list.flush()
    i = 0
    for mail in mail_list:
        entry = np.zeros((len(table)))
        for word in mail:
            if word in table.keys():
                entry[table[word]] += 1
        embedded_list[i,:] = entry
        i += 1
        if i % 100 == 0:
            embedded_list.flush()
    return(np.array(embedded_list))

In [9]:
mail_list, labels = get_mail_list(['datasets/spam/spam.bz2', 'datasets/spam/easy_ham.bz2'], [1,0])
table = tokenize(generate_word_list(mail_list))

In [11]:
import sys
sys.getsizeof(table)

961280

In [10]:
embedded = gen_embedded_matr(table, mail_list)

In [15]:
from sklearn.random_projection import GaussianRandomProjection as GRP
grp = GRP(random_state = 42)
reduced = grp.fit_transform(embedded)

MemoryError: Unable to allocate 8.39 GiB for an array with shape (214186, 10521) and data type float32

In [24]:
len(mail_list)

214186

In [ ]:
D_frame = pd.DataFrame(embedded)

In [ ]:
D_frame['label'] = labels

In [ ]:
from sklearn.model_selection import train_test_split
train, test =  train_test_split(D_frame, test_size = 0.2, stratify = D_frame['label'], random_state = 41)

In [ ]:
y_train = train['label']

In [ ]:
X_train = train.drop('label', axis =1)

In [ ]:
X_test, y_test =  test.drop('label', axis = 1), test['label']

In [ ]:
"""from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
KNC = KNeighborsClassifier(n_neighbors = 10)
KNC.fit(X_train, y_train)"""


from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier(random_state = 42)
SGD.fit(X_train,y_train)

"""RNC = RadiusNeighborsClassifier(radius = 10000)
RNC.fit(X_train, y_train)
RNC.predict([X_test.iloc[0]])"""

In [ ]:
SGD.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(SGD, X_train, y_train, cv = 5, scoring = 'accuracy')

In [ ]:
coeffs = SGD.coef_[0]
words = np.array(word_list[1:])

In [ ]:
words[coeffs.argsort()[-40:]]

In [ ]:
var = embed.T.dot(embed)

In [ ]:
centrd = embed - embed.mean(axis=0)

In [ ]:
u, e , vt = np.linalg.svd(centrd)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
red = pca.fit_transform(X_train)

In [ ]:
pca.n_components_

In [ ]:
SGD1 = SGDClassifier(random_state = 42)
SGD1.fit(red, labels)

In [13]:
filename = 'm.mmap'
testmap = np.memmap(filename, dtype = 'float32', mode = 'write', shape = (2,3))

In [14]:
len(table.keys())

27719

In [15]:
testmap[1] = [4,5,6]

In [14]:
embed

NameError: name 'embed' is not defined